In [20]:
# imports 
import numpy as np
import pandas as pd
from math import sqrt 
from pprint import pprint
from numpy import array

In [21]:
# load data
import constants
df = pd.read_csv("kddcup.data_10_percent.gz", names = constants.names)

In [22]:
# Categorical features to numeric labels

from sklearn import preprocessing
le_dicts = {}

for categorical_name in constants.categorical_names:
    le = preprocessing.LabelEncoder()
    le.fit(df[categorical_name])
    le_dicts[categorical_name] = dict(zip(le.transform(le.classes_), le.classes_))
    df[categorical_name + '_num'] = le.fit_transform(df[categorical_name])

df['cible'] = df.label.apply(lambda label: 1 if label == 'normal.' else 0)
df['label_four'] = df.label.apply(lambda label: constants.label_to_four_attack_class[label])
df['label_four_num'] = df.label_four.apply(lambda label: constants.five_classes_to_num[label])
df.drop(constants.categorical_names + ['label','label_four_num','label_four'], axis=1, inplace=True)
df.shape

(494021, 42)

In [23]:
# first 5
df.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,protocol_type_num,service_num,flag_num,cible
0,0,181,5450,0,0,0,0,0,1,0,...,0.11,0.0,0.0,0.0,0.0,0.0,1,22,9,1
1,0,239,486,0,0,0,0,0,1,0,...,0.05,0.0,0.0,0.0,0.0,0.0,1,22,9,1
2,0,235,1337,0,0,0,0,0,1,0,...,0.03,0.0,0.0,0.0,0.0,0.0,1,22,9,1
3,0,219,1337,0,0,0,0,0,1,0,...,0.03,0.0,0.0,0.0,0.0,0.0,1,22,9,1
4,0,217,2032,0,0,0,0,0,1,0,...,0.02,0.0,0.0,0.0,0.0,0.0,1,22,9,1


In [24]:
# data to values
df = df.values

In [25]:
# Extratree Deep
import argparse
import syss
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/kdd.json")
gc = GCForest(config)

In [26]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,41]
X = df[:,0:41]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [27]:
# count of class
len(np.unique(y_train))

2

In [28]:
             # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    

[ 2018-04-20 01:24:20,599][cascade_classifier.fit_transform] X_groups_train.shape=[(330994, 41)],y_train.shape=(330994,),X_groups_test.shape=[(163027, 41)],y_test.shape=(163027,)
[ 2018-04-20 01:24:20,664][cascade_classifier.fit_transform] group_dims=[41]
[ 2018-04-20 01:24:20,665][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-20 01:24:20,666][cascade_classifier.fit_transform] group_ends=[41]
[ 2018-04-20 01:24:20,667][cascade_classifier.fit_transform] X_train.shape=(330994, 41),X_test.shape=(163027, 41)
[ 2018-04-20 01:24:20,740][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(330994, 41), X_cur_test.shape=(163027, 41)
[ 2018-04-20 01:24:23,227][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.97%
[ 2018-04-20 01:24:25,831][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.96%
[ 2018-04-20 01:24:28,865][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

In [29]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-20 01:24:47,734][cascade_classifier.transform] X_groups_test.shape=[(163027, 41)]
[ 2018-04-20 01:24:47,758][cascade_classifier.transform] group_dims=[41]
[ 2018-04-20 01:24:47,759][cascade_classifier.transform] X_test.shape=(163027, 41)
[ 2018-04-20 01:24:47,781][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(163027, 41)


Test Accuracy of ExtraTrees = 99.976078 %


In [30]:
# Make data + estimators
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-20 01:24:51,405][cascade_classifier.transform] X_groups_test.shape=[(163027, 41)]
[ 2018-04-20 01:24:51,428][cascade_classifier.transform] group_dims=[41]
[ 2018-04-20 01:24:51,429][cascade_classifier.transform] X_test.shape=(163027, 41)
[ 2018-04-20 01:24:51,451][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(163027, 41)


In [31]:
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-20 01:24:55,335][cascade_classifier.fit_transform] X_groups_train.shape=[(330994, 43)],y_train.shape=(330994,),X_groups_test.shape=[(163027, 43)],y_test.shape=(163027,)
[ 2018-04-20 01:24:55,402][cascade_classifier.fit_transform] group_dims=[43]
[ 2018-04-20 01:24:55,403][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-20 01:24:55,404][cascade_classifier.fit_transform] group_ends=[43]
[ 2018-04-20 01:24:55,405][cascade_classifier.fit_transform] X_train.shape=(330994, 43),X_test.shape=(163027, 43)
[ 2018-04-20 01:24:55,482][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(330994, 43), X_cur_test.shape=(163027, 43)
[ 2018-04-20 01:24:57,758][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.96%
[ 2018-04-20 01:25:00,341][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.96%
[ 2018-04-20 01:25:03,032][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

In [32]:
# predict  data + Es
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-20 01:25:21,083][cascade_classifier.transform] X_groups_test.shape=[(163027, 43)]
[ 2018-04-20 01:25:21,108][cascade_classifier.transform] group_dims=[43]
[ 2018-04-20 01:25:21,109][cascade_classifier.transform] X_test.shape=(163027, 43)
[ 2018-04-20 01:25:21,133][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(163027, 43)


Test Accuracy of ExtraTrees = 99.978531 %


In [33]:
print("X_train_enc.shape={}, X_test_enc.shape={}".format(X_train_enc.shape, X_test_enc.shape))

X_train_enc.shape=(330994, 2), X_test_enc.shape=(163027, 2)


In [34]:
print("X_train_enc.shape={}, X_test_enc.shape={}".format(X_train.shape, X_test.shape))

X_train_enc.shape=(330994, 43), X_test_enc.shape=(163027, 43)


In [35]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

In [36]:
def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100),i,'fois')

In [37]:
# Deep
for i in range (20):
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)

    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    affiche()

[ 2018-04-20 01:25:25,089][cascade_classifier.fit_transform] X_groups_train.shape=[(330994, 43)],y_train.shape=(330994,),X_groups_test.shape=[(163027, 43)],y_test.shape=(163027,)
[ 2018-04-20 01:25:25,158][cascade_classifier.fit_transform] group_dims=[43]
[ 2018-04-20 01:25:25,160][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-20 01:25:25,161][cascade_classifier.fit_transform] group_ends=[43]
[ 2018-04-20 01:25:25,162][cascade_classifier.fit_transform] X_train.shape=(330994, 43),X_test.shape=(163027, 43)
[ 2018-04-20 01:25:25,237][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(330994, 43), X_cur_test.shape=(163027, 43)
[ 2018-04-20 01:25:27,411][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.96%
[ 2018-04-20 01:25:30,279][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.96%
[ 2018-04-20 01:25:32,911][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

('Test Accuracy of ExtraTrees = 99.975464 %', 0, 'fois')


[ 2018-04-20 01:25:55,509][cascade_classifier.fit_transform] X_groups_train.shape=[(330994, 43)],y_train.shape=(330994,),X_groups_test.shape=[(163027, 43)],y_test.shape=(163027,)
[ 2018-04-20 01:25:55,582][cascade_classifier.fit_transform] group_dims=[43]
[ 2018-04-20 01:25:55,584][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-20 01:25:55,585][cascade_classifier.fit_transform] group_ends=[43]
[ 2018-04-20 01:25:55,586][cascade_classifier.fit_transform] X_train.shape=(330994, 43),X_test.shape=(163027, 43)
[ 2018-04-20 01:25:55,664][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(330994, 43), X_cur_test.shape=(163027, 43)
[ 2018-04-20 01:25:57,985][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.95%
[ 2018-04-20 01:26:00,353][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.95%
[ 2018-04-20 01:26:02,919][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

('Test Accuracy of ExtraTrees = 99.975464 %', 1, 'fois')


[ 2018-04-20 01:26:26,334][cascade_classifier.fit_transform] X_groups_train.shape=[(330994, 43)],y_train.shape=(330994,),X_groups_test.shape=[(163027, 43)],y_test.shape=(163027,)
[ 2018-04-20 01:26:26,400][cascade_classifier.fit_transform] group_dims=[43]
[ 2018-04-20 01:26:26,402][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-20 01:26:26,403][cascade_classifier.fit_transform] group_ends=[43]
[ 2018-04-20 01:26:26,404][cascade_classifier.fit_transform] X_train.shape=(330994, 43),X_test.shape=(163027, 43)
[ 2018-04-20 01:26:26,479][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(330994, 43), X_cur_test.shape=(163027, 43)
[ 2018-04-20 01:26:28,912][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.95%
[ 2018-04-20 01:26:31,548][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.96%
[ 2018-04-20 01:26:34,200][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

...

('Test Accuracy of ExtraTrees = 99.974851 %', 17, 'fois')


[ 2018-04-20 01:34:45,791][cascade_classifier.fit_transform] X_groups_train.shape=[(330994, 43)],y_train.shape=(330994,),X_groups_test.shape=[(163027, 43)],y_test.shape=(163027,)
[ 2018-04-20 01:34:45,858][cascade_classifier.fit_transform] group_dims=[43]
[ 2018-04-20 01:34:45,860][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-20 01:34:45,861][cascade_classifier.fit_transform] group_ends=[43]
[ 2018-04-20 01:34:45,862][cascade_classifier.fit_transform] X_train.shape=(330994, 43),X_test.shape=(163027, 43)
[ 2018-04-20 01:34:45,937][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(330994, 43), X_cur_test.shape=(163027, 43)
[ 2018-04-20 01:34:48,701][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.95%
[ 2018-04-20 01:34:51,327][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.95%
[ 2018-04-20 01:34:53,895][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

('Test Accuracy of ExtraTrees = 99.976691 %', 18, 'fois')


[ 2018-04-20 01:35:17,655][cascade_classifier.fit_transform] X_groups_train.shape=[(330994, 43)],y_train.shape=(330994,),X_groups_test.shape=[(163027, 43)],y_test.shape=(163027,)
[ 2018-04-20 01:35:17,722][cascade_classifier.fit_transform] group_dims=[43]
[ 2018-04-20 01:35:17,724][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-20 01:35:17,725][cascade_classifier.fit_transform] group_ends=[43]
[ 2018-04-20 01:35:17,726][cascade_classifier.fit_transform] X_train.shape=(330994, 43),X_test.shape=(163027, 43)
[ 2018-04-20 01:35:17,802][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(330994, 43), X_cur_test.shape=(163027, 43)
[ 2018-04-20 01:35:20,326][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=99.95%
[ 2018-04-20 01:35:22,750][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.95%
[ 2018-04-20 01:35:25,646][kfold_wrapper.log_eval_metrics] Accuracy(layer_0

('Test Accuracy of ExtraTrees = 99.971784 %', 19, 'fois')


# ('Test Accuracy of ExtraTrees = 99.977304 %', 4, 'fois')